# A Simple URL Classifer

The following notebook contains code to build a simple URL classifier from training to testing it with both CPU and GPU support models. Just dump the dataset, tweak the code and run, learn on the go!

Made by: Sumit Gupta (steosumit)

## Import the libraries

In [30]:
import pandas as pd
#from oracle_automl import AutoML
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


## Download and load the dataset

In [31]:
PATH = "D:\Work\Projects\LinkShield\data\dataset_phishing.csv"  # Change this accordingly
df = pd.read_csv(PATH)
df.describe()

,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,...,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank
count,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.0,11430.000000,...,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,1.143000e+04,11430.000000,11430.000000,11430.000000
mean,61.126684,21.090289,0.150569,2.480752,0.997550,0.022222,0.141207,0.162292,0.0,0.293176,...,0.124759,0.775853,0.439545,0.072878,492.532196,4062.543745,8.567566e+05,0.020122,0.533946,3.185739
std,55.297318,10.777171,0.357644,1.369686,2.087087,0.155500,0.364456,0.821337,0.0,0.998317,...,0.330460,0.417038,0.496353,0.259948,814.769415,3107.784600,1.995606e+06,0.140425,0.498868,2.536955
min,12.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,-1.000000,-12.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,33.000000,15.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,1.000000,0.000000,0.000000,84.000000,972.250000,0.000000e+00,0.000000,0.000000,1.000000
50%,47.000000,19.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,1.000000,0.000000,0.000000,242.000000,3993.000000,1.651000e+03,0.000000,1.000000,3.000000
75%,71.000000,24.000000,0.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,1.000000,1.000000,0.000000,449.000000,7026.750000,3.738455e+05,0.000000,1.000000,5.000000
max,1641.000000,214.000000,1.000000,24.000000,43.000000,4.000000,3.000000,19.000000,0.0,19.000000,...,1.000000,1.000000,1.000000,1.000000,29829.000000,12874.000000,1.076799e+07,1.000000,1.000000,10.000000


## Preprocessing

In [32]:
# Select the features
features = [
    'length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq',
    'nb_underscore', 'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma', 'nb_semicolumn',
    'nb_dollar', 'nb_space', 'nb_www', 'nb_com', 'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url',
    'ratio_digits_host', 'punycode', 'shortening_service', 'path_extension', 'phish_hints', 'domain_in_brand',
    'brand_in_subdomain', 'brand_in_path', 'suspecious_tld'
]
numerical_df = df.select_dtypes('float64', 'int64')
corr_matrix = numerical_df.corr()
for s in corr_matrix:
    print(s)

ratio_digits_url
ratio_digits_host
avg_words_raw
avg_word_host
avg_word_path
ratio_intHyperlinks
ratio_extHyperlinks
ratio_extRedirection
ratio_extErrors
links_in_tags
ratio_intMedia
ratio_extMedia
safe_anchor


## Feature Engineering : Correlation based feature selection

In [33]:
"""Developer: Do not iterate of the corr_matrix directly, do it column wise as or states corr scores of one column with others"""
def feature_selector_correlation(cmatrix, target_cols, threshold=0.1):
    selected_features = {}
    for target_col in target_cols:
        correlations = cmatrix[target_col]  # correlation of all features with the target
        selected_features = {}
        for feature, score in correlations.items():
            if abs(score) >= threshold and feature != target_col:
                selected_features[feature] = score
    return selected_features
selected_features = list((feature_selector_correlation(corr_matrix, numerical_df.columns, threshold=0.01)).keys())

In [39]:
# Training code
## Splitting the dataset
X = df[selected_features]
y = df['status']

# Encode the target variable (convert 'legitimate'/'phishing' to 0/1)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.25, random_state=42)

## Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
"""
My code ended here, the rest is a AI assisted coding content, so please so have extra watch
"""


Original labels: ['legitimate' 'phishing']
Encoded labels: ['legitimate' 'phishing']
Mapping: {'legitimate': np.int64(0), 'phishing': np.int64(1)}

✓ Label encoder and scaler saved!


## Training the model using AutoML (Oracle Propreitary Library)

We will be using the Oracle AutoML library to train the simple classifier that runs various model training algorithms and state the best performing models

### AutoMLx: Making model training function by automlx

In [10]:
from automlx import init, Pipeline
def train_phishing_classifier(X: pd.DataFrame,
                              y: pd.Series,
                              test_size: float = 0.2,
                              random_state: int = 42,
                              task: str = 'classification',
                              **automlx_kwargs):
    """
    Train a classifier using Oracle AutoMLx on the supplied DataFrame.
    Returns the trained pipeline/model and test performance report.
    """
    # 3. Initialize AutoMLx engine (optional customization)
    init(engine='local')  # you may pass n_jobs, etc.
    # 4. Create AutoMLx pipeline
    pipeline = Pipeline(task=task, random_state=random_state, **automlx_kwargs)
    # 5. Fit the pipeline
    pipeline.fit(X, y)
    # 6. Make predictions on test set
    y_pred = pipeline.predict(X_test_scaled)
    # 7. Evaluate
    report = classification_report(y_test, y_pred, output_dict=True)

    return pipeline, report

train_phishing_classifier(X_train_scaled, y_train, test_size=0.25, random_state=42, task='classification')


ModuleNotFoundError: No module named 'automlx'

## Manual Approach: making the model training function by manually specifying the model

### Adding CPU models

In [40]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import joblib

USE_GPU = True  # Change to False to use CPU only

# Step 1: Make a model type list, adding models manually based on the situation( both GPU and CPU)
model_type_list = {}

# Add sklearn models (CPU only)
model_type_list.update({
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000),
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'KNN': KNeighborsClassifier(),
    'GaussianNB': GaussianNB()
})

print(f"\nGPU + CPU Total models: {len(model_type_list)}")
print(f"Models: {list(model_type_list.keys())}\n")


✓ XGBoost with GPU enabled
✓ LightGBM with GPU enabled

✓ Total models: 10
Models: ['XGBoost', 'LightGBM', 'LogisticRegression', 'DecisionTree', 'RandomForest', 'GradientBoosting', 'AdaBoost', 'SVM', 'KNN', 'GaussianNB']

Training: XGBoost
Training with default parameters...

Test Set Performance:
  Accuracy:  0.8555
  Precision: 0.8556
  Recall:    0.8555
  F1-Score:  0.8555
Training: LightGBM
Training with default parameters...

Test Set Performance:
  Accuracy:  0.8481
  Precision: 0.8482
  Recall:    0.8481
  F1-Score:  0.8481
Training: LogisticRegression
Performing Grid Search with 3-fold CV...
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
Best CV score: 0.6893

Test Set Performance:
  Accuracy:  0.7043
  Precision: 0.7105
  Recall:    0.7043
  F1-Score:  0.7024
Training: DecisionTree
Performing Grid Search with 3-fold CV...
Fitting 3 folds for each of 90 candidates, totalling 270 fits
Best parameters: {'

KeyboardInterrupt: 

### Adding CPU parameter grids

In [ ]:
# Step 2: Make a parameter list with respect to each model type
parameter_list = {
    'LogisticRegression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l2'],
        'solver': ['lbfgs', 'liblinear']
    },
    'DecisionTree': {
        'max_depth': [5, 10, 15, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'criterion': ['gini', 'entropy']
    },
    'RandomForest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'criterion': ['gini', 'entropy']
    },
    'GradientBoosting': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10]
    },
    'AdaBoost': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.5, 1.0]
    },
    'SVM': {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto']
    },
    'KNN': {
        'n_neighbors': [3, 5, 7, 9, 11],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan', 'minkowski']
    },
    'GaussianNB': {
        'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]
    }
}

### CPU Training Function

In [ ]:
# Commented out: Original manually wriiten CPU only training function
def train_manual_classifier(X_train, y_train, X_test, y_test,
                            model_type_list, parameter_list,
                            cv=3):
    results = {}
    best_score = 0
    best_model = None
    best_model_name = None

    for model_name, model in model_type_list.items():
        print(f"Training: {model_name}")
        try:
            if model_name in parameter_list:
                print(f"Performing Grid Search with {cv}-fold CV...")
                grid_search = GridSearchCV(
                    estimator=model,
                    param_grid=parameter_list[model_name],
                    cv=cv,
                    scoring='accuracy',
                    n_jobs=-1,
                    verbose=1
                )
                grid_search.fit(X_train, y_train)
                trained_model = grid_search.best_estimator_  # the best estimator
                print(f"Best parameters: {grid_search.best_params_}")  # the best parameters of the best estimator
                print(f"Best CV score: {grid_search.best_score_:.4f}")  # the best score from grid search
            else:
                print("Training with default parameters...")
                trained_model = model
                trained_model.fit(X_train, y_train)

            # Make predictions
            y_pred = trained_model.predict(X_test)

            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
            recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
            f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

            # Store results
            results[model_name] = {
                'model': trained_model,
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1_score': f1,
                'predictions': y_pred
            }

            # Print metrics
            print(f"\nTest Set Performance:")
            print(f"  Accuracy:  {accuracy:.4f}")
            print(f"  Precision: {precision:.4f}")
            print(f"  Recall:    {recall:.4f}")
            print(f"  F1-Score:  {f1:.4f}")

            # Track best model
            if accuracy > best_score:
                best_score = accuracy
                best_model = trained_model
                best_model_name = model_name

        except Exception as e:
            print(f"Error training {model_name}: {str(e)}")
            results[model_name] = {
                'error': str(e)
            }

    print(f"BEST MODEL: {best_model_name}")
    print(f"BEST ACCURACY: {best_score:.4f}")
    print(f"{'='*80}")

    return results, best_model, best_model_name

# Run the function
results_tuned, best_model_tuned, best_model_name_tuned = train_manual_classifier(
    X_train_scaled, y_train,
    X_test_scaled, y_test,
    model_type_list, parameter_list,
)

# Display results summary
print("\n" + "="*80)
print("RESULTS SUMMARY")
print("="*80)
results_df = pd.DataFrame({
    model_name: {
        'Accuracy': metrics.get('accuracy', 0),
        'Precision': metrics.get('precision', 0),
        'Recall': metrics.get('recall', 0),
        'F1-Score': metrics.get('f1_score', 0)
    }
    for model_name, metrics in results_tuned.items() if 'accuracy' in metrics
}).T.sort_values('Accuracy', ascending=False)

print(results_df)

# Print detailed classification report for best model
print(f"\n{'='*80}")
print(f"DETAILED CLASSIFICATION REPORT - {best_model_name_tuned}")
print(f"{'='*80}")
print(classification_report(y_test, results_tuned[best_model_name_tuned]['predictions'],
                          target_names=label_encoder.classes_))

### Adding GPU models

In [41]:
import warnings
warnings.filterwarnings('ignore')
import time

# Import GPU-accelerated libraries
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
    print("✓ XGBoost available")
except ImportError:
    XGBOOST_AVAILABLE = False
    print("✗ XGBoost not available - install with: pip install xgboost")

try:
    import lightgbm as lgb
    LIGHTGBM_AVAILABLE = True
    print("✓ LightGBM available")
except ImportError:
    LIGHTGBM_AVAILABLE = False
    print("✗ LightGBM not available - install with: pip install lightgbm")

try:
    from cuml.ensemble import RandomForestClassifier as cuRF
    from cuml.linear_model import LogisticRegression as cuLR
    from cuml.svm import SVC as cuSVC
    from cuml.neighbors import KNeighborsClassifier as cuKNN
    import cupy as cp
    CUML_AVAILABLE = True
    print("✓ CuML (RAPIDS) available - GPU acceleration enabled!")
except ImportError:
    CUML_AVAILABLE = False
    print("ℹ CuML not available - using CPU fallback")
    print("  XGBoost and LightGBM will still use GPU acceleration!")
    print("  CuML is optional - only install if you need additional GPU models")
    print("")
    print("  Note: CuML on Windows can be challenging to install. Weird I struggled with AI Agent too")

print("\n" + "="*80)

# Step 1: Make GPU-accelerated model type list
model_type_list_gpu = {}

# Add GPU-accelerated models if available
if XGBOOST_AVAILABLE:
    print("Adding XGBoost GPU models...")
    model_type_list_gpu['XGBoost_GPU'] = xgb.XGBClassifier(
        device='cuda',  # GPU acceleration
        tree_method='hist',
        random_state=42,
        n_jobs=-1
    )
    model_type_list_gpu['XGBoost_CPU'] = xgb.XGBClassifier(
        device='cpu',
        tree_method='hist',
        random_state=42,
        n_jobs=-1
    )

if LIGHTGBM_AVAILABLE:
    print("Adding LightGBM GPU models...")
    model_type_list_gpu['LightGBM_GPU'] = lgb.LGBMClassifier(
        device='gpu',  # GPU acceleration
        random_state=42,
        n_jobs=-1,
        verbose=-1
    )
    model_type_list_gpu['LightGBM_CPU'] = lgb.LGBMClassifier(
        device='cpu',
        random_state=42,
        n_jobs=-1,
        verbose=-1
    )

if CUML_AVAILABLE:
    print("Adding CuML GPU models...")
    # CuML models (RAPIDS) - native GPU support
    model_type_list_gpu['RandomForest_GPU'] = cuRF(n_estimators=100, random_state=42)
    model_type_list_gpu['LogisticRegression_GPU'] = cuLR(random_state=42, max_iter=1000)
    model_type_list_gpu['SVM_GPU'] = cuSVC(random_state=42)
    model_type_list_gpu['KNN_GPU'] = cuKNN(n_neighbors=5)
else:
    # Fallback to CPU models from sklearn
    print("Adding CPU fallback models...")
    model_type_list_gpu['RandomForest_CPU'] = RandomForestClassifier(random_state=42, n_jobs=-1, n_estimators=100)
    model_type_list_gpu['LogisticRegression_CPU'] = LogisticRegression(random_state=42, max_iter=1000)
    model_type_list_gpu['SVM_CPU'] = SVC(random_state=42)
    model_type_list_gpu['KNN_CPU'] = KNeighborsClassifier(n_jobs=-1)

# CPU models for comparison
model_type_list_gpu['DecisionTree'] = DecisionTreeClassifier(random_state=42)
model_type_list_gpu['GradientBoosting'] = GradientBoostingClassifier(random_state=42)
model_type_list_gpu['AdaBoost'] = AdaBoostClassifier(random_state=42)
model_type_list_gpu['GaussianNB'] = GaussianNB()

print(f"Total models to train(GPU Focused): {len(model_type_list_gpu)}")
print(f"Models: {list(model_type_list_gpu.keys())}")


✓ XGBoost available
✓ LightGBM available
ℹ CuML not available - using CPU fallback (THIS IS NORMAL)
  XGBoost and LightGBM will still use GPU acceleration!
  CuML is optional - only install if you need additional GPU models

  Note: CuML on Windows can be challenging to install.
  Your GPU training will work great with XGBoost and LightGBM!

Adding XGBoost GPU models...
Adding LightGBM GPU models...
Adding CPU fallback models...

✓ Total models to train: 12
Models: ['XGBoost_GPU', 'XGBoost_CPU', 'LightGBM_GPU', 'LightGBM_CPU', 'RandomForest_CPU', 'LogisticRegression_CPU', 'SVM_CPU', 'KNN_CPU', 'DecisionTree', 'GradientBoosting', 'AdaBoost', 'GaussianNB']


### Adding GPU parameter grids

In [42]:
# Step 2: Make parameter list for GPU models
parameter_list_gpu = {}

# XGBoost parameters (works for both GPU and CPU)
if XGBOOST_AVAILABLE:
    parameter_list_gpu['XGBoost_GPU'] = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.3],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
    parameter_list_gpu['XGBoost_CPU'] = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.3],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }

# LightGBM parameters (works for both GPU and CPU)
if LIGHTGBM_AVAILABLE:
    parameter_list_gpu['LightGBM_GPU'] = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7, -1],
        'learning_rate': [0.01, 0.1, 0.3],
        'num_leaves': [31, 50, 100],
        'subsample': [0.8, 1.0]
    }
    parameter_list_gpu['LightGBM_CPU'] = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7, -1],
        'learning_rate': [0.01, 0.1, 0.3],
        'num_leaves': [31, 50, 100],
        'subsample': [0.8, 1.0]
    }

# CuML/sklearn RandomForest parameters
if CUML_AVAILABLE:
    parameter_list_gpu['RandomForest_GPU'] = {
        'n_estimators': [50, 100, 200],
        'max_depth': [10, 20, 30],
        'max_features': [0.5, 0.8, 1.0]
    }
else:
    parameter_list_gpu['RandomForest_CPU'] = {
        'n_estimators': [50, 100, 200],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': [2, 5, 10],
        'criterion': ['gini', 'entropy']
    }

# Logistic Regression parameters
if CUML_AVAILABLE:
    parameter_list_gpu['LogisticRegression_GPU'] = {
        'C': [0.01, 0.1, 1, 10, 100],
        'max_iter': [1000, 2000]
    }
else:
    parameter_list_gpu['LogisticRegression_CPU'] = {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l2'],
        'solver': ['lbfgs', 'liblinear']
    }

# Other model parameters
parameter_list_gpu['DecisionTree'] = {
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}

parameter_list_gpu['GradientBoosting'] = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

parameter_list_gpu['AdaBoost'] = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.5, 1.0]
}

parameter_list_gpu['GaussianNB'] = {
    'var_smoothing': [1e-9, 1e-8, 1e-7]
}

print(f"✓ Parameter grids configured for {len(parameter_list_gpu)} models")


✓ Parameter grids configured for 10 models


### GPU Training Function

In [43]:
# Step 3: GPU-accelerated training function
def train_gpu_classifier(X_train, y_train, X_test, y_test,
                         model_type_list, parameter_list,
                         use_grid_search=False, cv=3):
    """
    Train multiple classifiers with GPU acceleration and optional hyperparameter tuning.

    Parameters:
    -----------
    X_train : array-like
        Training features
    y_train : array-like
        Training labels
    X_test : array-like
        Test features
    y_test : array-like
        Test labels
    model_type_list : dict
        Dictionary of model names and instances
    parameter_list : dict
        Dictionary of hyperparameter grids for each model
    use_grid_search : bool
        Whether to perform grid search for hyperparameter tuning
    cv : int
        Number of cross-validation folds for grid search

    Returns:
    --------
    results : dict
        Dictionary containing model performance metrics
    best_model : object
        The best performing model
    best_model_name : str
        Name of the best performing model
    """
    results = {}
    best_score = 0
    best_model = None
    best_model_name = None

    print("="*80)
    print("TRAINING MODELS (GPU-ACCELERATED)")
    print("="*80)

    for model_name, model in model_type_list.items():
        print(f"\n{'='*80}")
        print(f"Training: {model_name}")
        print(f"{'='*80}")

        start_time = time.time()

        try:
            # Convert data for CuML models (they need cupy arrays)
            if CUML_AVAILABLE and '_GPU' in model_name and 'XGBoost' not in model_name and 'LightGBM' not in model_name:
                X_train_gpu = cp.array(X_train)
                y_train_gpu = cp.array(y_train)
                X_test_gpu = cp.array(X_test)
                y_test_gpu = cp.array(y_test)
            else:
                X_train_gpu = X_train
                y_train_gpu = y_train
                X_test_gpu = X_test
                y_test_gpu = y_test

            if use_grid_search and model_name in parameter_list:
                print(f"Performing Grid Search with {cv}-fold CV...")
                grid_search = GridSearchCV(
                    estimator=model,
                    param_grid=parameter_list[model_name],
                    cv=cv,
                    scoring='accuracy',
                    n_jobs=-1 if 'GPU' not in model_name else 1,
                    verbose=1
                )
                grid_search.fit(X_train_gpu, y_train_gpu)
                trained_model = grid_search.best_estimator_
                print(f"Best parameters: {grid_search.best_params_}")
                print(f"Best CV score: {grid_search.best_score_:.4f}")
            else:
                print("Training with default parameters...")
                trained_model = model
                trained_model.fit(X_train_gpu, y_train_gpu)

            # Make predictions
            y_pred = trained_model.predict(X_test_gpu)

            # Convert back from GPU if needed
            if CUML_AVAILABLE and '_GPU' in model_name and 'XGBoost' not in model_name and 'LightGBM' not in model_name:
                y_pred = cp.asnumpy(y_pred)
                y_test_cpu = cp.asnumpy(y_test_gpu)
            else:
                y_test_cpu = y_test

            # Calculate metrics
            accuracy = accuracy_score(y_test_cpu, y_pred)
            precision = precision_score(y_test_cpu, y_pred, average='weighted', zero_division=0)
            recall = recall_score(y_test_cpu, y_pred, average='weighted', zero_division=0)
            f1 = f1_score(y_test_cpu, y_pred, average='weighted', zero_division=0)

            training_time = time.time() - start_time

            # Store results
            results[model_name] = {
                'model': trained_model,
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1_score': f1,
                'predictions': y_pred,
                'training_time': training_time
            }

            # Print metrics
            print(f"\nTest Set Performance:")
            print(f"  Accuracy:  {accuracy:.4f}")
            print(f"  Precision: {precision:.4f}")
            print(f"  Recall:    {recall:.4f}")
            print(f"  F1-Score:  {f1:.4f}")
            print(f"  Time:      {training_time:.2f}s {'🚀 (GPU)' if '_GPU' in model_name else '(CPU)'}")

            # Track best model
            if accuracy > best_score:
                best_score = accuracy
                best_model = trained_model
                best_model_name = model_name

        except Exception as e:
            print(f"❌ Error training {model_name}: {str(e)}")
            import traceback
            traceback.print_exc()
            results[model_name] = {
                'error': str(e)
            }

    print(f"\n{'='*80}")
    print(f"🏆 BEST MODEL: {best_model_name}")
    print(f"🎯 BEST ACCURACY: {best_score:.4f}")
    print(f"{'='*80}")

    return results, best_model, best_model_name


In [44]:
# Step 4: Train models with GPU acceleration
print("🚀 Starting GPU-accelerated training...")
print("\n💡 TIP: Open another terminal and run 'nvidia-smi -l 1' to monitor GPU usage\n")

# Train without grid search first (faster)
results_gpu, best_model_gpu, best_model_name_gpu = train_gpu_classifier(
    X_train_scaled, y_train,
    X_test_scaled, y_test,
    model_type_list_gpu, parameter_list_gpu,
    use_grid_search=False,  # Set to True for hyperparameter tuning (slower but better)
    cv=3
)

# Display results summary
print("\n" + "="*80)
print("📊 RESULTS SUMMARY (GPU vs CPU)")
print("="*80)
results_df_gpu = pd.DataFrame({
    model_name: {
        'Accuracy': metrics.get('accuracy', 0),
        'Precision': metrics.get('precision', 0),
        'Recall': metrics.get('recall', 0),
        'F1-Score': metrics.get('f1_score', 0),
        'Time (s)': metrics.get('training_time', 0)
    }
    for model_name, metrics in results_gpu.items() if 'accuracy' in metrics
}).T.sort_values('Accuracy', ascending=False)

print(results_df_gpu)

# Print detailed classification report for best model
print(f"\n{'='*80}")
print(f"📋 DETAILED CLASSIFICATION REPORT - {best_model_name_gpu}")
print(f"{'='*80}")
print(classification_report(y_test, results_gpu[best_model_name_gpu]['predictions'],
                          target_names=label_encoder.classes_))

# Save the best model
model_filename = f'url_classifier_{best_model_name_gpu}.pkl'
joblib.dump(best_model_gpu, model_filename)
print(f"\n✅ Best model saved as '{model_filename}'")

# Compare GPU vs CPU performance if both are available
if XGBOOST_AVAILABLE or LIGHTGBM_AVAILABLE:
    print("\n" + "="*80)
    print("⚡ GPU SPEEDUP COMPARISON")
    print("="*80)
    for model_name in results_gpu.keys():
        if '_GPU' in model_name and 'accuracy' in results_gpu[model_name]:
            cpu_model_name = model_name.replace('_GPU', '_CPU')
            if cpu_model_name in results_gpu and 'training_time' in results_gpu[cpu_model_name]:
                gpu_time = results_gpu[model_name]['training_time']
                cpu_time = results_gpu[cpu_model_name]['training_time']
                speedup = cpu_time / gpu_time if gpu_time > 0 else 0
                print(f"{model_name:25} GPU: {gpu_time:6.2f}s | CPU: {cpu_time:6.2f}s | Speedup: {speedup:.2f}x")


🚀 Starting GPU-accelerated training...

💡 TIP: Open another terminal and run 'nvidia-smi -l 1' to monitor GPU usage

TRAINING MODELS (GPU-ACCELERATED)

Training: XGBoost_GPU
Training with default parameters...

Test Set Performance:
  Accuracy:  0.8555
  Precision: 0.8556
  Recall:    0.8555
  F1-Score:  0.8555
  Time:      0.56s 🚀 (GPU)

Training: XGBoost_CPU
Training with default parameters...

Test Set Performance:
  Accuracy:  0.8558
  Precision: 0.8559
  Recall:    0.8558
  F1-Score:  0.8558
  Time:      0.26s (CPU)

Training: LightGBM_GPU
Training with default parameters...

Test Set Performance:
  Accuracy:  0.8481
  Precision: 0.8482
  Recall:    0.8481
  F1-Score:  0.8481
  Time:      3.74s 🚀 (GPU)

Training: LightGBM_CPU
Training with default parameters...

Test Set Performance:
  Accuracy:  0.8481
  Precision: 0.8482
  Recall:    0.8481
  F1-Score:  0.8481
  Time:      0.30s (CPU)

Training: RandomForest_CPU
Training with default parameters...

Test Set Performance:
  Accura

In [46]:
# Optional: Train with Grid Search for hyperparameter tuning (uncomment to use)
print("\n\n" + "="*80)
print("🔧 TRAINING WITH HYPERPARAMETER TUNING (This will take longer)")
print("="*80)

results_gpu_tuned, best_model_gpu_tuned, best_model_name_gpu_tuned = train_gpu_classifier(
    X_train_scaled, y_train,
    X_test_scaled, y_test,
    model_type_list_gpu, parameter_list_gpu,
    use_grid_search=True,  # Enable grid search
    cv=5
)

print("\n📊 TUNED RESULTS:")
results_df_tuned = pd.DataFrame({
    model_name: {
        'Accuracy': metrics.get('accuracy', 0),
        'Precision': metrics.get('precision', 0),
        'Recall': metrics.get('recall', 0),
        'F1-Score': metrics.get('f1_score', 0),
        'Time (s)': metrics.get('training_time', 0)
    }
    for model_name, metrics in results_gpu_tuned.items() if 'accuracy' in metrics
}).T.sort_values('Accuracy', ascending=False)

print(results_df_tuned)



🔧 TRAINING WITH HYPERPARAMETER TUNING (This will take longer)
TRAINING MODELS (GPU-ACCELERATED)

Training: XGBoost_GPU
Performing Grid Search with 5-fold CV...
Fitting 5 folds for each of 108 candidates, totalling 540 fits


KeyboardInterrupt: 